In [ ]:
import numpy as np
import math
import re
import random
import matplotlib.pyplot as plt

In [ ]:
CTR = "./CTR.txt"

with open(CTR, "r") as f:
    content = f.readlines()
    
content = [x.split(':') for x in content]

articles = [x[1].split(';') for x in content]
clicks = [x[2].split(';') for x in content]

articles = [[float(x_i) for x_i in x] for x in articles]
clicks = [[float(x_i) for x_i in x] for x in clicks]

clicks = np.array(clicks)
articles = np.array(articles)

# Methods

In [ ]:
class Strategy:
    def __init__(self, strat, articles, clicks, delta=0.05, alpha=None):
        self.strat = strat
        self.articles = articles
        self.clicks = clicks
        self.score = 0
        self.static_best = np.argmax(np.sum(clicks, axis=0))
        
        # for UCB
        self.reward_history = np.zeros(10)
        self.choice_history = np.zeros(10)
        
        # LinUCB
        self.A_list = [np.eye(5) for _ in range(10)]
        self.b_list = [np.zeros(5) for _ in range(10)]
        if alpha is None:
            self.alpha = 1 + np.sqrt(np.log(2 / delta) / 2)
        else:
            self.alpha = alpha
            
        # Thompson Sampling for Contextual Bandits (Agrawal & Goyal, 2013):
        # not applicable here, as we have a global context vector x_t all machines (announcers),
        # not individual context vectors x_it
        
        
    def select(self, t):
        if self.strat == 'Random':
            return random.randint(0,9)
        
        if self.strat == 'StaticBest':
            return self.static_best
        
        if self.strat == 'Optimal':
            return np.argmax(clicks[t])
        
        if self.strat == 'UCB':
            ub_estimate = [1/self.choice_history[i]*self.reward_history[i] 
                             + np.sqrt(2*np.log(t) / self.choice_history[i]) for i in range(10)]
            
            choice = np.argmax(ub_estimate)
            
            self.reward_history[choice] += self.clicks[t, choice]
            self.choice_history[choice] += 1
            return choice
        
        if 'LinUCB' in self.strat:
            x_t = self.articles[t]
            piche = np.zeros(10)
            
            for i in range(10):
                A = self.A_list[i]
                b = self.b_list[i]
                A_inv = np.linalg.inv(A)
                
                theta = A_inv.dot(b)
                piche[i] = theta.dot(x_t) + self.alpha * np.sqrt(x_t.dot(A_inv).dot(x_t))
                
            choice = np.argmax(piche)
            r_t = self.clicks[t, choice]

            self.A_list[choice] += x_t.dot(x_t)
            self.b_list[choice] += r_t * x_t
            return choice
        
        if 'Thompson' in self.strat:
            x_t = self.articles[t]
            
            # Sample a theta
            A_inv = np.linalg.inv(self.A)
            theta = np.random.multivariate_normal(self.b.dot(A_inv), A_inv)
            
            # Find the arm that maximizes our reward expectancy
            reward_expectancies = [theta.dot(x_t) for _ in]

                
        
    def reward(self, t):
        return clicks[t][self.select(t)]


In [ ]:
articles[2]

In [ ]:
strat_names = ['Random', 'StaticBest', 'Optimal', 'UCB', 'LinUCB-0.15']
strategies = {}
for strat in strat_names:
    if 'LinUCB' in strat:
        alpha = float(strat.split('-')[1])
        strategies[strat] = Strategy(strat, articles, clicks, alpha=alpha)
    else:
        strategies[strat] = Strategy(strat, articles, clicks)


# Baselines & UCB

In [ ]:
total_reward = []
total_regret = []

for t in range(len(clicks)):
    rewards = []
    regrets = []
    r_t_staticbest = clicks[t, strategies['StaticBest'].select(t)]
    
    for strat in strat_names:
        i = strategies[strat].select(t)
        r_ti = clicks[t, i]
        rewards.append(r_ti)
        regrets.append(r_t_staticbest - r_ti)
        
    total_reward.append(rewards)
    total_regret.append(regrets)
    
total_regret = np.cumsum(np.array(total_regret), axis=0)
total_reward = np.cumsum(np.array(total_reward), axis=0)

In [ ]:
plt.figure(figsize=(15, 15))

for k, s in enumerate(strat_names):
    plt.plot(total_reward[:, k], label=s)

plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15, 15))

for k, s in enumerate(strat_names):
    plt.plot(total_regret[:, k], label=s)

plt.legend()
plt.show()

# Contextual Bandits : Lin-UCB

In [ ]:
def get_final_regret(alphas):
    lin_ucb_strategies = [Strategy('LinUCB', articles, clicks, alpha=a) for a in alphas]

    total_reward = []
    total_regret = []

    for t in range(len(clicks)):
        if t%1000 == 0:
            print(f'{t}/{len(clicks)}')
        rewards = []
        regrets = []
        r_t_staticbest = clicks[t, strategies['StaticBest'].select(t)]

        for strat in lin_ucb_strategies:
            i = strat.select(t)
            r_ti = clicks[t, i]
            rewards.append(r_ti)
            regrets.append(r_t_staticbest - r_ti)

        total_reward.append(rewards)
        total_regret.append(regrets)

    total_regret = np.cumsum(np.array(total_regret), axis=0)
    total_reward = np.cumsum(np.array(total_reward), axis=0)

    return total_regret[-1, :]

Bounds that make sense for alpha in a statistical setting (with a 1-$\delta$ confidence level)

In [ ]:
deltas = np.logspace(-2.5, 0, num=20)
alphas = np.array([1 + np.sqrt(np.log(2 / delta) / 2) for delta in deltas])
plt.plot(deltas, alphas)
plt.xlabel('delta')
plt.ylabel('alpha')
plt.show()

In [ ]:
final_regret = get_final_regret(alphas)

In [ ]:
plt.figure(figsize=(12,12))
plt.subplot(221)
plt.plot(alphas, final_regret)
plt.ylabel('final regret')
plt.xlabel('alpha')
plt.subplot(222)
plt.plot(deltas, final_regret)
plt.ylabel('final regret')
plt.xlabel('delta')
plt.show()

But in reality we can tune alpha directly to go further in both directions:
- bigger delta = smaller confidence intervals, i.e. more confidence in our estimates => smaller alpha & less exploration
- smaller delta/bigger alpha => more exploration

Let's explore the hyperparameter space

In [ ]:
alphas = np.logspace(-2, 2, num=100)
final_regret = get_final_regret(alphas)

In [ ]:
plt.plot(alphas, final_regret)
plt.ylabel('final regret')
plt.xlabel('alpha')
plt.xscale('log')
plt.show()

In [ ]:
alphas = np.logspace(-1, -0.5, num=200)
final_regret = get_final_regret(alphas)

In [ ]:
plt.plot(alphas, final_regret)
plt.ylabel('final regret')
plt.xlabel('alpha')
plt.xscale('log')
plt.show()

In [ ]:
i = np.argmin(final_regret)
print(f'Best alpha: {alphas[i]}, final regret: {final_regret[i]} (run {i})')